In [1]:
import torch

In [6]:
newone = torch.load("experiments/TRAIN/MetaController/checkpoints/last.pth")

In [3]:
ported = torch.load("experiments/TRAIN/MetaController_ported/checkpoints/step:200000.pth")

In [4]:
ported.keys()

dict_keys(['state_dict', 'config', 'global_step'])

In [7]:
newone.keys()

dict_keys(['state_dict', 'config', 'global_step'])

In [18]:
config_ported = newone['config']
setattr(config_ported, 'val_iter', 10000)
setattr(config_ported, 'global_batch_size', 64)

In [21]:
config_ported

{'seed': 0,
 'precision': 'bf16',
 'strategy': 'ddp',
 'gradient_clip_val': 5.0,
 'env_name': None,
 'num_train_trajs': -1,
 'num_eval_trajs': 3,
 'shot': 4,
 'history_size': 10,
 'window_size': 10,
 'max_n_joints': 10,
 'num_workers': 8,
 'global_batch_size': 64,
 'num_eval_episodes': 8,
 'hidden_dim': 512,
 'n_morph_blocks': 6,
 'n_task_blocks': 6,
 'n_attn_heads': 4,
 'morphology_lora_rank': 16,
 'task_lora_rank': 16,
 'morphology_layerscale': 1.0,
 'task_layerscale': 1.0,
 'drop_rate': 0.1,
 'n_steps': 200000,
 'n_schedule_steps': 200000,
 'optimizer': 'adamw',
 'lr': 0.0002,
 'lr_schedule': 'poly',
 'lr_warmup': 1000,
 'lr_warmup_scale': 0.0,
 'schedule_from': 0,
 'weight_decay': 0.0,
 'lr_decay_degree': 0.9,
 'log_dir': 'TRAIN',
 'save_dir': 'TRAIN',
 'load_dir': 'TRAIN',
 'val_iter': 10000,
 'save_interval': 1,
 'monitor': 'summary/mtrain_test',
 'monitor_mode': 'max',
 'load_step': -1,
 'config_path': 'configs/train_config.yaml',
 'continue_mode': False,
 'no_eval': False,
 're

In [54]:
def custom_indexing_task(task):
    if task in DMC.ENVIRONMENTS:
        t_idx = DMC.ENVIRONMENTS.index(task)
    elif task[0] in ['hopper_longshort', 'hopper_shortlong', 'hopper_smallfoot', 'hopper_largefoot']:
        t_idx = DMC.ENVIRONMENTS.index(('hopper', 'hop'))
    elif task[0] in ['wolf_longshort', 'wolf_shortlong']:
        t_idx = DMC.ENVIRONMENTS.index(('wolf', 'walk'))
    elif task[0] in ['reacher_four_longfront', 'reacher_four_longmid']:
        t_idx = DMC.ENVIRONMENTS.index(('reacher_four', 'hard'))
    return t_idx

def custom_indexing_morphology(morphology):
    if morphology in DMC.MORPHOLOGIES:
        m_idx = DMC.MORPHOLOGIES.index(morphology)
    elif morphology in ['hopper_longshort', 'hopper_shortlong', 'hopper_smallfoot', 'hopper_largefoot']:
        m_idx = DMC.MORPHOLOGIES.index('hopper')
    elif morphology in ['wolf_longshort', 'wolf_shortlong']:
        m_idx = DMC.MORPHOLOGIES.index('wolf')
    elif morphology in ['reacher_four_longfront', 'reacher_four_longmid']:
        m_idx = DMC.MORPHOLOGIES.index('reacher_four')   
    return m_idx

class DMC(object):
    NAME = 'dmc' 
    CUSTOM_WALKERS = True
    ADDITIONAL_ENVS = True
    ENVIRONMENTS = [
        ('acrobot', 'swingup'),
        ('cartpole', 'balance'),
        ('cartpole', 'swingup'),
        ('cartpole_three', 'poles'),
        ('cartpole_two', 'poles'),
        ('cheetah', 'flip'),
        ('cheetah', 'flip_backwards'),
        ('cheetah', 'jump'),
        ('cheetah', 'legs_up'),
        ('cheetah', 'lie_down'),
        ('cheetah', 'run'),
        ('cheetah', 'run_back'),
        ('cheetah', 'run_backwards'),
        ('cheetah', 'run_front'),
        ('cheetah', 'stand_back'),
        ('cheetah', 'stand_front'),
        ('cup', 'catch'),
        ('cup', 'spin'),
        ('finger', 'spin'),
        ('hopper', 'flip'),
        ('hopper', 'flip_backwards'),
        ('hopper', 'hop'),
        ('hopper', 'hop_backwards'),
        ('hopper', 'stand'),
        ('pendulum', 'spin'),
        ('pendulum', 'swingup'),
        ('pointmass', 'easy'),
        ('reacher', 'hard'),
        ('reacher_four', 'hard'),
        ('reacher_three', 'hard'),
        ('walker', 'arabesque'),
        ('walker', 'backflip'),
        ('walker', 'flip'),
        ('walker', 'headstand'),
        ('walker', 'legs_up'),
        ('walker', 'lie_down'),
        ('walker', 'run'),
        ('walker', 'run_backwards'),
        ('walker', 'stand'),
        ('walker', 'walk'),
        ('walker', 'walk_backwards'),
    ]
    MORPHOLOGIES = sorted(list(set([m for m, _ in ENVIRONMENTS])))

    if CUSTOM_WALKERS:
        ENVIRONMENTS += [
            ('horse', 'run'),
            ('horse', 'walk'),
            ('kangaroo', 'run'),
            ('kangaroo', 'walk'),
            ('ostrich', 'run'),
            ('ostrich', 'walk'),
            ('wolf', 'run'),
            ('wolf', 'walk'),
        ]
        MORPHOLOGIES += [
            'horse',
            'kangaroo',
            'ostrich',
            'wolf'
        ]
    
    if ADDITIONAL_ENVS:
        ENVIRONMENTS += [
            ('finger', 'turn_easy'),
            ('finger', 'turn_hard'),
            ('reacher_four', 'easy')
        ]

    EVAL_ENVIRONMENTS = [
        ('acrobot', 'swingup'),
        ('cartpole_two', 'poles'),
        ('cheetah', 'run_backwards'),
        ('cup', 'catch'),
        ('pendulum', 'swingup'),
        ('pointmass', 'easy'),
        ('reacher_three', 'hard'),
        ('walker', 'run'),
    ]

    RANDOM_ENVIRONMENTS = [
        ('cheetah', 'random'),
        ('cheetah_fiveback', 'random'),
        ('cheetah_fivebalanced', 'random'),
        ('cheetah_fivefront', 'random'),
        ('cheetah_fourallback', 'random'),
        ('cheetah_fourallfront', 'random'),
        ('cheetah_fourback', 'random'),
        ('cheetah_fourfront', 'random'),
        ('cheetah_sixback', 'random'),
        ('cheetah_sixfront', 'random'),
        ('cheetah_threeback', 'random'),
        ('cheetah_threebalanced', 'random'),
        ('cheetah_threefront', 'random'),
        ('cheetah_twoback', 'random'),
        ('hopper', 'random'),
        ('hopper_four', 'random'),
        ('walker', 'random'),
        ('hopper_three', 'random'),
        ('walker_five', 'random'),
        ('walker_four', 'random'),
        ('walker_six', 'random'),
        ('walker_three', 'random'),
        ('walker_two', 'random')
    ]
    
    VARIANT_ENVIRONMENTS = [
        ('hopper_largefoot', 'stand'),
        ('hopper_smallfoot', 'stand'),
        ('hopper_longshort', 'stand'),
        ('hopper_shortlong', 'stand'),
        ('wolf_longshort', 'walk'),
        ('wolf_shortlong', 'walk'),
        ('reacher_four_longfront', 'easy'),
        ('reacher_four_longmid', 'easy')
    ]

In [65]:
NEW_ENVIRONMENTS = [
        ('acrobot', 'swingup'),
        ('cartpole', 'balance'),
        ('cartpole', 'swingup'),
        ('cartpole_two', 'poles'),
        ('cheetah', 'flip'),
        ('cheetah', 'flip_backwards'),
        ('cheetah', 'jump'),
        ('cheetah', 'legs_up'),
        ('cheetah', 'lie_down'),
        ('cheetah', 'run'),
        ('cheetah', 'run_back'),
        ('cheetah', 'run_backwards'),
        ('cheetah', 'run_front'),
        ('cheetah', 'stand_back'),
        ('cheetah', 'stand_front'),
        ('cup', 'catch'),
        ('cup', 'spin'),
        ('hopper', 'hop'),
        ('hopper', 'hop_backwards'),
        ('hopper', 'stand'),
        ('pendulum', 'spin'),
        ('pendulum', 'swingup'),
        ('pointmass', 'easy'),
        ('reacher', 'hard'),
        ('reacher_three', 'hard'),
        ('reacher_four', 'easy'),
        ('reacher_four', 'hard'),
        ('walker', 'arabesque'),
        ('walker', 'backflip'),
        ('walker', 'flip'),
        ('walker', 'headstand'),
        ('walker', 'legs_up'),
        ('walker', 'lie_down'),
        ('walker', 'run'),
        ('walker', 'stand'),
        ('walker', 'walk'),
        ('walker', 'walk_backwards'),
        ('wolf', 'run'),
        ('wolf', 'walk')
    ]
NEW_MORPHOLOGIES = sorted(list(set([m for m, _ in NEW_ENVIRONMENTS])))

['model.obs_morphology_transformer.position_embedding.2',
 'model.obs_morphology_transformer.position_embedding.3',
 'model.obs_morphology_transformer.position_embedding.4',
 'model.obs_morphology_transformer.position_embedding.5',
 'model.obs_morphology_transformer.position_embedding.6',
 'model.obs_morphology_transformer.position_embedding.7',
 'model.obs_morphology_transformer.position_embedding.8',
 'model.obs_morphology_transformer.position_embedding.9',
 'model.obs_morphology_transformer.position_embedding.10',
 'model.obs_morphology_transformer.position_embedding.11',
 'model.obs_morphology_transformer.position_embedding.12',
 'model.obs_morphology_transformer.obs_morphology_encoder.blocks.0.norm1.bias',
 'model.obs_morphology_transformer.obs_morphology_encoder.blocks.0.attn.q.bias',
 'model.obs_morphology_transformer.obs_morphology_encoder.blocks.0.attn.q.lora_A',
 'model.obs_morphology_transformer.obs_morphology_encoder.blocks.0.attn.q.lora_B',
 'model.obs_morphology_transform

In [60]:
ported['state_dict']['model.obs_morphology_transformer.position_embedding.2'].shape

torch.Size([1, 4, 512])

In [57]:
newone['state_dict']['model.obs_morphology_transformer.position_embedding.2'].shape

torch.Size([1, 3, 512])

In [62]:
old_only = [] 
new_only = []
shape_mis = []
others = []

for key in ported['state_dict']:
    if key not in newone['state_dict']:
        old_only.append(key)
    
    elif ported['state_dict'][key].shape != newone['state_dict'][key].shape:
        shape_mis.append(key)
        
for key in newone['state_dict']:
    if key not in ported['state_dict']:
        new_only.append(key)

for key in ported['state_dict']:
    if key not in old_only + new_only + shape_mis:
        others.append(key)

In [61]:
cleanse_state_dict = {}

In [63]:
for key in others:
    cleanse_state_dict[key] = ported['state_dict'][key]

for key 

In [49]:
old_only

['model.obs_morphology_transformer.obs_embedding.proj_global.6.weight',
 'model.obs_morphology_transformer.obs_embedding.proj_global.6.bias',
 'model.obs_morphology_transformer.obs_embedding.proj_global.7.weight',
 'model.obs_morphology_transformer.obs_embedding.proj_global.7.bias',
 'model.obs_morphology_transformer.obs_embedding.proj_global.11.weight',
 'model.obs_morphology_transformer.obs_embedding.proj_global.11.bias',
 'model.obs_morphology_transformer.obs_embedding.proj_global.12.weight',
 'model.obs_morphology_transformer.obs_embedding.proj_global.12.bias',
 'model.obs_morphology_transformer.position_embedding.13',
 'model.obs_morphology_transformer.position_embedding.14',
 'model.obs_morphology_transformer.position_embedding.15',
 'model.obs_morphology_transformer.position_embedding.16',
 'model.obs_morphology_transformer.position_embedding.17']

In [50]:
new_only

['model.obs_morphology_transformer.obs_embedding.proj_global.5.weight',
 'model.obs_morphology_transformer.obs_embedding.proj_global.5.bias',
 'model.obs_morphology_transformer.obs_embedding.proj_global.8.weight',
 'model.obs_morphology_transformer.obs_embedding.proj_global.8.bias',
 'model.obs_morphology_transformer.obs_embedding.proj_global.9.weight',
 'model.obs_morphology_transformer.obs_embedding.proj_global.9.bias']

In [51]:
shape_mis

['model.obs_morphology_transformer.position_embedding.2',
 'model.obs_morphology_transformer.position_embedding.3',
 'model.obs_morphology_transformer.position_embedding.4',
 'model.obs_morphology_transformer.position_embedding.5',
 'model.obs_morphology_transformer.position_embedding.6',
 'model.obs_morphology_transformer.position_embedding.7',
 'model.obs_morphology_transformer.position_embedding.8',
 'model.obs_morphology_transformer.position_embedding.9',
 'model.obs_morphology_transformer.position_embedding.10',
 'model.obs_morphology_transformer.position_embedding.11',
 'model.obs_morphology_transformer.position_embedding.12',
 'model.obs_morphology_transformer.obs_morphology_encoder.blocks.0.norm1.bias',
 'model.obs_morphology_transformer.obs_morphology_encoder.blocks.0.attn.q.bias',
 'model.obs_morphology_transformer.obs_morphology_encoder.blocks.0.attn.q.lora_A',
 'model.obs_morphology_transformer.obs_morphology_encoder.blocks.0.attn.q.lora_B',
 'model.obs_morphology_transform